In [1]:
import tensorflow as tf
import utils
import os
import glob

from dataset import get_datasets
from stormer import Stormer

2024-04-22 16:14:57.179424: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-22 16:14:57.204820: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-22 16:14:57.639515: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/zuherj/miniconda3/envs/kws/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_

In [2]:
# print model names

print_model_table = lambda model_list: utils.print_enumerated_list(model_list, "Model")

models_names = [path.split("/")[-1] for path in glob.glob("models/stormer*")]
models_names.sort()
print_model_table(models_names)
model_name = models_names[int(input("Choose model: "))]

Available Models:
-------------------------------------------------
| Index        | Model Name                     |
|--------------|--------------------------------|
| 0            | stormer_r2_h4_dm32_dataset=mel |
|______________|________________________________|


In [3]:
hps = utils.load_hps(model_name)
stormer = Stormer(**hps)

2024-04-22 16:15:02.488027: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-04-22 16:15:02.504224: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-04-22 16:15:02.504350: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [4]:
# check if the model containing directory exists
model_path = utils.get_model_path(model_name)
load_weights = os.path.exists(os.path.dirname(model_path))
if load_weights:
    stormer.load_weights(model_path)

In [5]:
## load the datasets
train, valid, test = get_datasets(
    **hps
)

Loading dataset version 2
Dataset loaded


2024-04-22 16:15:03.747863: I tensorflow_io/core/kernels/cpu_check.cc:128] Your CPU supports instructions that this TensorFlow IO binary was not compiled to use: AVX AVX2 FMA


In [6]:
# for layer in stormer.layers[:-1]:
#     print(layer.name)
#     layer.trainable = False

In [7]:
results_filename = f'data/results/{model_name}.csv'

metrics=["accuracy"]

stormer.compile(
    optimizer=tf.keras.optimizers.AdamW(hps["learning_rate"]),
    loss="categorical_crossentropy",
    metrics=metrics,
)

model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=model_path,
    save_weights_only=True,
    save_freq="epoch",
    verbose=0,
)

state_transformer_history = stormer.fit(
    train,
    validation_data=valid,
    epochs=hps["num_epochs"],
    callbacks=[
        model_checkpoint_callback,
        utils.MetricsLogger(
            results_filename,
        )
    ],
)

Epoch 1/10000


2024-04-22 16:15:28.642057: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:606] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2024-04-22 16:15:28.709020: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7ff11400f180 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-04-22 16:15:28.709036: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA RTX A5000, Compute Capability 8.6
2024-04-22 16:15:28.713028: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-04-22 16:15:28.802528: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8600
2024-04-22 16:15:28.862890: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.

1489/1489 [==============================] - 193s 109ms/step - loss: 3.5629 - accuracy: 0.3402 - val_loss: 2.8196 - val_accuracy: 0.5377
Epoch 2/10000
1489/1489 [==============================] - 165s 109ms/step - loss: 2.8348 - accuracy: 0.5197 - val_loss: 2.2672 - val_accuracy: 0.6727
Epoch 3/10000
 142/1489 [=>............................] - ETA: 2:24 - loss: 2.5848 - accuracy: 0.5819

KeyboardInterrupt: 